In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas
import pandas as pd
from IPython.display import display, HTML

In [17]:
# Critere de performance
def compute_pred_score(y_true, y_pred):
    y_pred_unq =  np.unique(y_pred)
    for i in y_pred_unq:
        if((i != -1) & (i!= 1) & (i!= 0) ):
            raise ValueError('The predictions can contain only -1, 1, or 0!')
    y_comp = y_true * y_pred
    score = float(10*np.sum(y_comp == -1) + np.sum(y_comp == 0))
    score /= y_comp.shape[0]
    return score

In [18]:
X_train_fname = 'training_templates.csv'
y_train_fname = 'training_labels.txt'
X_test_fname  = 'testing_templates.csv'
y_train = np.loadtxt(y_train_fname, dtype=np.int)
X_train = pd.read_csv(X_train_fname, sep=',', header=None).values
X_test  = pd.read_csv(X_test_fname,  sep=',', header=None).values
print X_train.shape
print y_train.shape
print X_test.shape

(105600, 128)
(105600,)
(8496, 128)


### Preprocessing

In [19]:
# def reject_outliers(data, result, m=2):
#     return data[abs(data - np.mean(data)) < m * np.std(data)],result[abs(data - np.mean(data)) < m * np.std(data)]
# X_outliers, y_outliers = reject_outliers(X_train, y_train)
# print X_outliers.shape
# print y_outliers.shape

In [32]:
# KNN
from sklearn.neighbors import NearestNeighbors

# Standardization
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

# Feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
SKB = SelectKBest(mutual_info_classif, k=115)
X_train_skb = SKB.fit_transform(X_train_std, y_train)
X_test_skb = SKB.transform(X_test_std)

# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=115, whiten=True)
X_train_pca = pca.fit_transform(X_train_skb)
X_test_pca = pca.transform(X_test_skb)

# split data into train and test sets
from sklearn.cross_validation import train_test_split
seed = 0
test_size = 0.08
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train_pca, y_train, 
                                                                test_size=test_size, random_state=seed, stratify=y_train)
print X_train_split.shape, y_train_split.shape
print X_test_split.shape, y_test_split.shape
print np.mean(y_train_split)

(97152, 115) (97152,)
(8448, 115) (8448,)
0.0


### Training

In [33]:
# -1 ? 0 ? 1
def predict(y_predict_proba, threshold=0.86):
    y_predict = np.zeros((len(y_predict_proba)), dtype=np.int)
    y_diff = np.add(y_predict_proba[:,1], -y_predict_proba[:,0])
    y_predict = np.piecewise(y_diff,[y_diff<(-threshold),y_diff>threshold],[-1,1]).astype(int)
    return y_predict

### MLP

In [34]:
import time
start = time.clock()
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(20,5,2), random_state=1)
clf.fit(X_train_split, y_train_split)
y_inter = clf.predict_proba(X_test_split)

# To select a best Threshold:
score_min = 1
threshold = 0.7
for x in np.linspace(0.5, 0.99, num=50):
    y_pred_train = predict(y_inter,x)
    score = compute_pred_score(y_test_split, y_pred_train)
    if score < score_min:
        score_min = score
        threshold = x
    print('Threshold %s \t Score : %s' %(x,score))

# Play with the best threshold
y_pred_train = predict(y_inter,threshold)
score = compute_pred_score(y_test_split, y_pred_train)
print('Score sur le train : %s' % score)
end = time.clock()
print "Running Time: ",end-start

Threshold 0.5 	 Score : 0.178858901515
Threshold 0.51 	 Score : 0.178858901515
Threshold 0.52 	 Score : 0.178148674242
Threshold 0.53 	 Score : 0.177675189394
Threshold 0.54 	 Score : 0.176728219697
Threshold 0.55 	 Score : 0.176846590909
Threshold 0.56 	 Score : 0.175071022727
Threshold 0.57 	 Score : 0.175544507576
Threshold 0.58 	 Score : 0.173650568182
Threshold 0.59 	 Score : 0.169152462121
Threshold 0.6 	 Score : 0.168323863636
Threshold 0.61 	 Score : 0.167140151515
Threshold 0.62 	 Score : 0.16631155303
Threshold 0.63 	 Score : 0.164299242424
Threshold 0.64 	 Score : 0.163707386364
Threshold 0.65 	 Score : 0.16181344697
Threshold 0.66 	 Score : 0.159919507576
Threshold 0.67 	 Score : 0.160511363636
Threshold 0.68 	 Score : 0.161221590909
Threshold 0.69 	 Score : 0.159446022727
Threshold 0.7 	 Score : 0.15790719697
Threshold 0.71 	 Score : 0.15849905303
Threshold 0.72 	 Score : 0.157078598485
Threshold 0.73 	 Score : 0.154474431818
Threshold 0.74 	 Score : 0.154237689394
Thresho

In [ ]:
import time
start = time.clock()
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(20,5,2), random_state=1)
clf.fit(X_train_pca, y_train)
y_inter = clf.predict_proba(X_test_pca)

# Play with the best threshold
y_pred_train = predict(y_inter,threshold)
y_pred = predict(y_inter,0.6)
np.savetxt('y_pred.txt', y_pred, fmt='%d')
end = time.clock()
print "Running Time: ",end-start

### Bagging

In [13]:
import time
start = time.clock()
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
n_estimators = 40
# clf = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(20,5,2), random_state=1)
# clf.fit(X_train_split, y_train_split)
bagging = BaggingClassifier(base_estimator=MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(20,5,2), random_state=27),
                           n_estimators=n_estimators,
                           random_state=1, max_samples=0.8)
bagging.fit(X_train_split, y_train_split)
y_inter = bagging.predict_proba(X_test_split)

# To select a best Threshold:
score_min = 1
threshold = 0.7
for x in np.linspace(0.5, 0.99, num=50):
    y_pred_train = predict(y_inter,x)
    score = compute_pred_score(y_test_split, y_pred_train)
    if score < score_min:
        score_min = score
        threshold = x
    print('Threshold %s \t Score : %s' %(x,score))

# Play with the best threshold
y_pred_train = predict(y_inter,threshold)
score = compute_pred_score(y_test_split, y_pred_train)
print('Score sur le train : %s' % score)
end = time.clock()
print "Running Time: ",end-start

Threshold 0.5 	 Score : 0.0864109848485
Threshold 0.51 	 Score : 0.0862926136364
Threshold 0.52 	 Score : 0.086884469697
Threshold 0.53 	 Score : 0.0873579545455
Threshold 0.54 	 Score : 0.0871212121212
Threshold 0.55 	 Score : 0.0845170454545
Threshold 0.56 	 Score : 0.0857007575758
Threshold 0.57 	 Score : 0.0848721590909
Threshold 0.58 	 Score : 0.0817945075758
Threshold 0.59 	 Score : 0.0828598484848
Threshold 0.6 	 Score : 0.0814393939394
Threshold 0.61 	 Score : 0.0813210227273
Threshold 0.62 	 Score : 0.0800189393939
Threshold 0.63 	 Score : 0.0801373106061
Threshold 0.64 	 Score : 0.0813210227273
Threshold 0.65 	 Score : 0.0826231060606
Threshold 0.66 	 Score : 0.0840435606061
Threshold 0.67 	 Score : 0.0825047348485
Threshold 0.68 	 Score : 0.0833333333333
Threshold 0.69 	 Score : 0.0823863636364
Threshold 0.7 	 Score : 0.0830965909091
Threshold 0.71 	 Score : 0.0864109848485
Threshold 0.72 	 Score : 0.0881865530303
Threshold 0.73 	 Score : 0.0892518939394
Threshold 0.74 	 Sco

In [14]:
import time
start = time.clock()
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
n_estimators = 40
clf = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(20,5,2), random_state=1)
clf.fit(X_train_pca, y_train)
bagging = BaggingClassifier(base_estimator=clf,
                           n_estimators=n_estimators,
                           random_state=1, max_samples=0.8)
bagging.fit(X_train_pca, y_train)
y_inter = bagging.predict_proba(X_test_pca)

# Play with the best threshold
y_pred_train = predict(y_inter,threshold)
y_pred = predict(y_inter,0.62)
np.savetxt('y_pred_bagging.txt', y_pred, fmt='%d')
end = time.clock()
print "Running Time: ",end-start

Running Time:  1604.565103
